# WhisperTS joining JSON to SRT

In [1]:
import re,codecs,unidecode
import pandas as pd
import networkx as nx
import matplotlib.pylab as p

import json,glob,os
nMachine=os.uname()[1]
if "iMac" in nMachine:
    nMachine="iMac"
elif "MacBook" in nMachine:
    nMachine="MacBook"
print(nMachine)

iMac


In [2]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

In [3]:
repJson="/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/%s/Json/"%nMachine
segments=len(glob.glob(repJson+"*.json"))
segments

8

In [4]:
json1=repJson+"extrait-00.json"
with open(json1) as inFile:
    whisper1=json.load(inFile)

### Gestion des timestamps

In [5]:
def second2timestamp(seconds):
    sHours=int(seconds/3600)
    sMinutes=int((seconds-sHours*3600)/60)
    sSeconds=int((seconds-sHours*3600-sMinutes*60))
    sMilliseconds=int((seconds%1)*1000)
    return "%02d:%02d:%02d,%03d"%(sHours,sMinutes,sSeconds,sMilliseconds)

def timestamp2second(timestamp):
    m=re.match("(\d\d):(\d\d):(\d\d),(\d\d\d)",timestamp)
    if m:
        seconds=(int(m.group(1))*60+int(m.group(2)))*60+int(m.group(3))+float(m.group(4))/1000
        return seconds
    else:
        print("problème de timestamp")
        return

def shiftTime(timestamp,decal):
    return second2timestamp(timestamp2second(timestamp)+decal)

In [6]:
punct = list('''!()[]{};:-'"\,<>./?@#$%^&*_~''')

def removePunct(texte):
    result=texte
    for p in punct:
        result=result.replace(p,"")
    return result

### Gestion des structures JSON de Whisper

In [7]:
def getWords(whisper):
    lWords=[]
    for segment in whisper["segments"]:
        for word in segment["words"]:
            lWords.append((second2timestamp(word["start"]),
                              second2timestamp(word["end"]),
                              word["text"],
                              word["confidence"]))        
    return [(t1,t2,w,c) for t1,t2,w,c in lWords if w!=""]

def getList(tWhisper):
    return [unidecode.unidecode(removePunct(w)).lower() for _,_,w,_ in tWhisper]

In [8]:
for s in whisper1['segments']:
    for w in s["words"]:
        if "start" not in w:
            print(w)

In [9]:
srt=getWords(whisper1)
srtList=getList(srt)

### Trouver la jointure

In [10]:
def findSample(sample,ref):
    lowSample=[t.lower() for t in sample]
    lowRef=[t.lower() for t in ref]
    i=0
    while i<len(ref)-len(sample):
        # print("ref",i)
        if lowSample==lowRef[i:i+len(sample)]:
            return i
        else:
            i+=1
    return None

### Collage des mots
1. on cherche la partie commune entre le nouveau segment et les sous-titres précédents
2. on peut régler la longueur de la partie commune pour optimiser le collage *maxSize*
3. on incrémente les temps de la nouvelle partie à partir du deuxième mot (pour éviter les erreurs de timestamp en début d'enregistrement)

In [11]:
def pasteWords(srt,whisper2):
    srtList=getList(srt)
    w2=getWords(whisper2)
    w2List=getList(w2)
    lenW2=len(w2)
    compSize=min(maxSize,lenW2)
    i=1
    print(compSize)
    while i<lenW2-compSize:
        sample=w2List[i:i+compSize]
        ref=srtList
        # print(sample)
        idx=findSample(sample,ref)
        if idx:
            print("found",i,idx)
            break
        else:
            i+=1
    if idx:
        print("found",i,idx,srtList[idx],srt[idx][0],w2List[i],w2[i][0])
        tDecal=timestamp2second(srt[idx+1][0])-timestamp2second(w2[i+1][0])
        print(second2timestamp(tDecal))
        newSrt=[]
        for n in range(i+1,lenW2):
            nWord=w2[n]
            newWord=(shiftTime(nWord[0],tDecal),shiftTime(nWord[1],tDecal),nWord[2],nWord[3])
            newSrt.append(newWord)
        print(srt[idx+1],newSrt[0])
        return srt[:idx]+newSrt
    else:
        print("problème",w2List)
        ding()
        ding()
        ding()
        return None

In [13]:
maxSize=8
if segments:
    for i in range(1,segments):
        print(i)
        json2=repJson+"extrait-%02d.json"%i
        with open(json2) as inFile:
            whisper2=json.load(inFile)
        newSrt=pasteWords(srt,whisper2)
        if not newSrt:
            break
        else:
            srt=newSrt
# print(srt)

1
8
found 2 220
found 2 220 cest 00:01:00,530 cest 00:00:00,500
00:01:00,029
('00:01:00,730', '00:01:00,929', 'vraiment', 0.784) ('00:01:00,729', '00:01:00,929', 'vraiment', 0.784)
2
8
found 29 511
found 29 511 jamais 00:02:33,169 jamais 00:00:33,009
00:02:00,058
('00:02:33,329', '00:02:33,489', 'de', 0.984) ('00:02:33,329', '00:02:33,549', 'de', 0.813)
3
8
found 48 626
found 48 626 bonheur 00:03:18,548 bonheur 00:00:18,929
00:03:00,057
('00:03:19,387', '00:03:19,449', 'est', 0.724) ('00:03:19,387', '00:03:19,447', 'est', 0.953)
4
8
found 1 780
found 1 780 ca 00:04:00,337 ca 00:00:00,280
00:04:00,037
('00:04:00,417', '00:04:00,536', 'doit', 0.999) ('00:04:00,417', '00:04:00,516', 'doit', 0.998)
5
8
found 5 959
found 5 959 vraiment 00:05:01,045 vraiment 00:00:01,340
00:05:00,028
('00:05:01,487', '00:05:02,167', 'si', 0.961) ('00:05:01,487', '00:05:02,168', 'si', 0.981)
6
8
found 7 1143
found 7 1143 sensuel 00:06:02,948 sensuel 00:00:02,720
00:05:59,849
('00:06:04,148', '00:06:04,407', '

In [14]:
pause=0.200

srtTours=[]
srtMots=[]
bTour=True
prevEnd=0
pStart=second2timestamp(0)
pEnd=0

for i,(start,end,mot,confidence) in enumerate(srt):
    # print(start,end,mot,confidence)
    tStart=timestamp2second(start)
    tEnd=timestamp2second(end)
    # print(start,tStart,prevEnd)
    if tStart-prevEnd>pause and prevEnd!=0:
        pEnd=second2timestamp(prevEnd)
        pTour=" ".join(srtMots)
        srtTours.append((pStart,pEnd,pTour))
        pStart=start
        prevEnd=tEnd
        srtMots=[mot]
        bPhrase=True
    else:
        srtMots.append(mot)
        prevEnd=tEnd
pTour=" ".join(srtMots)
srtTours.append((pStart,pEnd,pTour))

In [15]:
ding()

In [16]:
srtLines=[]
for i,(start,end,mot) in enumerate(srtTours):
    print(start,end,mot)
    n=i+1
    srtLines.append(str(n))
    srtLines.append(start+" --> "+end)
    srtLines.append(mot)
    srtLines.append("")


00:00:00,000 00:00:02,338 qui joue au Beach Soccer [*]
00:00:02,640 00:00:04,780 et ses visites sur Pathé Sport.
00:00:05,179 00:00:05,639 [*]
00:00:06,089 00:00:12,890 Bon alors le Beach Soccer, parlez-nous de ça, parce que c'est vrai qu'en France, dans le monde, c'est connu. En France, c'est moins connu. [*] Donc d'abord, qui y a dans l'équipe ?
00:00:13,529 00:00:16,030 [*] Aujourd'hui, il y a des mecs qui ont arrêté il n'y a pas longtemps,
00:00:16,289 00:00:18,289 [*] comme Fournier, comme Guéré.
00:00:18,530 00:00:19,268 Et Olmeta ?
00:00:19,530 00:00:41,250 Pascal [*] Olmeta, mon frère Joël et puis [*] d'autres moins connus qui ont été de [*] bons footballeurs. Mais notre ambition, à nous, c'est de [*] se [*] servir de tous ces joueurs comme ça, pour [*] que chacun développe dans sa région vraiment un sport où les jeunes jouent, qu'on [*] mette des [*] terrains à disposition des [*] jeunes, parce que c'est intérêt. On joue à pieds nus, ça ne coûte rien. Oui, on joue à 5 contre 5

In [17]:
with codecs.open(json1.replace(".json","-tours.srt"),"w",encoding="utf8") as outFile:
    for srtLine in srtLines:
        outFile.write(srtLine+"\n")